# Data Extraction


### 1. Get the Dataset

##### Download the Zip File from the link Given below:

https://unearthed-exploresa.s3-ap-southeast-2.amazonaws.com/Unearthed_5_SARIG_Data_Package.zip

#### 2. Extract the files in a desired folder
Open this jupyter-notebook from that folder

In [ ]:
# Importing the required libraries
import pandas as pd
import numpy as np

We'll be using the `sarig_rs_chem_exp.csv` file for preparing our dataset. Since the size of file is around ~11 GB, we'll specify the columns that we need for analysis. We'll also be specifying the low_memory parameter.
The columns we need are ->
- 'LONGITUDE_GDA94'
- 'LATITUDE_GDA94'
- 'CHEM_CODE'
- 'VALUE'
- 'UNIT'

If you wish to have a full look at the dataset, you can always remove the `usecols` parameter. Though, its highly likely that the kernel will crash. _Hack - You can specify nrows = 10 or 100 to preview the first few rows._ 

In [ ]:
# Mention either the complete or relative path of the sarig_rs_chem_exp.csv file
path = '/home/xavian/Downloads/The_Gawler_Challenge/GeoChem_Data/Unearthed_5_SARIG_Data_Package/SARIG_Data_Package2_Exported20052020/sarig_rs_chem_exp.csv'
data = pd.read_csv(path, low_memory=False, encoding = "cp1252", usecols = ['LONGITUDE_GDA94','LATITUDE_GDA94','CHEM_CODE', 'VALUE', 'UNIT'])

In [ ]:
# View the first few rows of the dataset
data.head()

In [ ]:
# Let's view all the unique UNITS
data.UNIT.unique()

In [ ]:
data.info()

In [ ]:
# We'll be removing all the non-numeric characters -> '<>-'
data['VALUE'] = data.VALUE.str.lstrip('<->')

In [ ]:
# We'll be dropping all the values with AMOUNT less than 10
data=data.drop(data[data['VALUE']=='0-10'].index)

In [ ]:
# Convert the datatype to numeric
data[['VALUE']] = data[['VALUE']].apply(pd.to_numeric)

In [ ]:
# Let's preview
data

In [ ]:
# Create a new column - UNIT_PPM by converting all the other units to PPM
data['UNIT_PPM'] = np.where(data.UNIT == 'ppb', data.VALUE/1000, (np.where(data.UNIT == '%', data.VALUE*10000, data.VALUE)))

In [ ]:
# Let's drop the UNIT & VALUE Column
data = data.drop(['UNIT', 'VALUE'], axis=1)

In [ ]:
df=data.reset_index()

In [ ]:
df

In [ ]:
# These are the Chem codes we'll be dropping 
drop_min=[ 'LOI', 'H2O_plus', 'H2O_minus', 'Insol', 'Total',
       'GoI', 'TOT/C', 'pH', 'EC', 'RADBK',
       'RADTC', 'HMIN', 'H2O','CPS_gamma']

In [ ]:
# Let's drop them!
for i in drop_min:
    print(i)
    df=df.drop(df[ df['CHEM_CODE'] == i].index)

In [ ]:
df.CHEM_CODE.unique()

In [ ]:
df

Make sure you have `lxml` installed for the next step. It's not installed if you are in a conda environment. Install it using `pip install lxml`

In [ ]:
# To select unique chem codes at every individual location, we'll be calculating VALUE = UNIT_PPM * PRICE per KG
link = "https://en.wikipedia.org/wiki/Prices_of_chemical_elements"
tables = pd.read_html(link,header=0)[0]

In [ ]:
# create new column names
table = tables[['Symbol', 'Name', 'Price[5]']]
table=table.drop(index=0)
table=table.rename(columns={"Price[5]": "Price in USD/Kg"})

In [ ]:
# View the table
table

In [ ]:
# Lets make a dictionary with symbols as keys and price as Value
s=table['Symbol'].to_list()
p=table['Price in USD/Kg'].to_list()
price_dict = {s[i]: p[i] for i in range(len(s))}

# Now lets replace the price with a range, with a fixed value. For Ex
price = {'Ti': '11.7','K': '13.6','Na': '3.43', 'Ca': '2.35', 'Ru': '10600',  'Zr': '37.1', 'Ge': '1010', 'Ir': '56200', 'Sr': '6.68', 'Ta': '312', 'Re': '4150','Nb': '85.6','La': '4.92', 'Li': '85.6','Ce': '4.71','As': '1.31', 'Ba': '0.275','Ti': '11.7', 'V': '385','F': '2.16'}
price.update(price)

# Lets check the Dictionary
print(price_dict)

In [ ]:
chemcode_list=df.CHEM_CODE.unique().tolist()

In [ ]:
# Lets find the CHEM_CODE whose value we don't know[which are not in the 'price' dict]
def uncommon(lst1, lst2): 
    lst3 = [value for value in lst1 if value not in lst2] 
    return lst3 
lst1 = s
lst2 = a
other=uncommon(lst1, lst2)
print(other) 

In [ ]:
# Now lets make a dict which will replace the 'other' CHEM_CODE with a specific element
element=['U', 'Si', 'Al', 'Ti', 'Fe', 'Mn', 'Mg', 'Ca', 'Na', 'K', 'P', 'Fe', ' ', 'Fe', 'Cr', 'V', 'Th', 'W', 'Ta', 'Nb', ' ', 'Na', 'Ba', 'Ca', ' ', 'Ca', 'Mg', 'Ca', ' ', '-', ' ', ' ', 'Fe', 'Fe', 'Ni', 'V', 'Zn', 'Sr', 'Cu', 'Zr', 'Hf', 'Sr', '-', 'K', 'Fe', 'Co', '-', 'Cl']
element_dict = {other[i]: element[i] for i in range(len(other))} 
print(element_dict)

In [ ]:
# Now lets make a new column so that we can make change without disturbing the original data
df[['CHEM_CODE_N']]=df[['CHEM_CODE']]

In [ ]:
# Now Lets replace the 'other' CHEM_CODE using the 'element_dict'
df=df.replace({"CHEM_CODE_N": element_dict})

In [ ]:
# Now lets drop some more dummy CHEM_CODE
# Doing it seperately reduces its asymptotic complexity
indexNames = df[ df['CHEM_CODE_N']==" " ].index
df=df.drop(indexNames)
indexNames = df[ df['CHEM_CODE_N']=="-" ].index
df=df.drop(indexNames)

In [ ]:
# Lets check the changes
df.CHEM_CODE_N.unique()

In [ ]:
# Now we will make find the Price of the chemcodes of column 'CHEM_CODE_N' and make a price column in dataframe 'df'
CHEM_CODE_N_price=[]
chemcode=df.CHEM_CODE_N.tolist()
for i in chemcode:
    CHEM_CODE_price.append(float(price[i]))   

df['price'] = np.array(CHEM_CODE_N_price)

In [ ]:
# Now we will make a new column VALUE with approx values of chemcodes
df['VALUE'] = df['UNIT_PPM']*df['price']

In [ ]:
# Now lets Drop the Unnesecary columns and reset its index
df=df.drop(columns=['price'])
df=df.reset_index()
df=df.drop(columns=['index'])

In [ ]:
# Now lets find the most valuable minerals at a point and drop the other less valueable ones[present at the same point]
most_valued_minerals_at_points=df.sort_values('VALUE', ascending=False).drop_duplicates(['LONGITUDE_GDA94','LATITUDE_GDA94']).reset_index()

By our Preliminary Testing and analysis of Datasets we found that 'most_valued_minerals_at_points' had a lot of Ca and hence was bias and not good for training. Hence we should trim some Ca's

In [ ]:
# Lets make a DataFrame with only Ca at different points
df_Ca=df.loc[df['CHEM_CODE']=='Ca'].drop_duplicates(['LONGITUDE_GDA94','LATITUDE_GDA94'])

In [ ]:
# Now we will sort them in descending order and make a new DataFrame with the top 100000 rows
df_Ca=df_Ca.sort_values('VALUE', ascending=False)
df_Ca=df_Ca.iloc[:100000,:]

In [ ]:
# Now we will make a index column by using reset_index() and using the index column to drop Ca from the main DataFrame
df_Ca=df_Ca.reset_index()
df=df.drop(df_Ca['index'].tolist())

In [ ]:
# Now we will create a final DataFrame which will be used for training
final_df = df.sort_values('VALUE', ascending=False).drop_duplicates(['LONGITUDE_GDA94','LATITUDE_GDA94']).reset_index()

In [ ]:
# Lets see how it looks
final_df

In [ ]:
# Now we will classify UNIT_PPM into three classes 
final_df.loc[final_df['UNIT_PPM']>50000, 'UNIT_PPM_CLASS']='HIGH'
final_df.loc[final_df['UNIT_PPM']<=50000, 'UNIT_PPM_CLASS']='MED'
final_df.loc[final_df['UNIT_PPM']<=100, 'UNIT_PPM_CLASS']='LOW'

In [ ]:
# Lets see how it looks
final_df

In [ ]:
# Finally lets Drop the unnescessary columns
final_df=final_df.drop(columns=['CHEM_CODE_N','index'])

In [ ]:
# Now finally lets export the DataFrame as csv so that we can use for training
final_df.to_csv('unsampled.csv')